In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import zipfile
import tensorflow as tf
import matplotlib.pyplot as plt
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
with zipfile.ZipFile("/kaggle/input/dogs-vs-cats/train.zip","r") as zip_ref:
    zip_ref.extractall("dogs-vs-cats")
    
with zipfile.ZipFile("/kaggle/input/dogs-vs-cats/test1.zip","r") as zip_ref:
    zip_ref.extractall("dogs-vs-cats")

In [ ]:
files = os.listdir('/kaggle/working/dogs-vs-cats/train')

In [ ]:
category = []
for file in files:
    if file.split('.')[0] == 'dog':
        category.append('dog')
    else:
        category.append('cat')
    
train_df = pd.DataFrame({'file': files,
                        'label': category})

In [ ]:
train_df.label.value_counts()

In [ ]:
img_generator_train = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255, validation_split = 0.3,
                                                                     rotation_range = 20, width_shift_range = 0.1,
                                                                     height_shift_range = 0.1, horizontal_flip = True
                                                                     )
img_generator_test = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)

In [ ]:
train_set = img_generator_train.flow_from_dataframe(dataframe=train_df, directory="/kaggle/working/dogs-vs-cats/train",
                                                    x_col="file", y_col='label', batch_size=32, class_mode = 'binary',
                                                    seed=42, shuffle=False, target_size=(256,256), subset="training")
val_set = img_generator_train.flow_from_dataframe(dataframe=train_df, directory="/kaggle/working/dogs-vs-cats/train",
                                                    x_col="file", y_col='label', batch_size=32, class_mode = 'binary',
                                                    seed=42, shuffle=False, target_size=(256,256), subset="validation")

In [ ]:
def plotImages(images_arr):
  fig, axes = plt.subplots(1, 5, figsize=(20,20))
  axes = axes.flatten()
  for img, ax in zip(images_arr, axes):
    ax.imshow(img)
  plt.tight_layout()
  plt.show()

In [ ]:
plotImages([train_set[0][0][1] for i in range(0,5)])

In [ ]:
test_df = pd.DataFrame({'file':os.listdir('/kaggle/working/dogs-vs-cats/test1')})
test = img_generator_test.flow_from_dataframe(dataframe=test_df, directory= '/kaggle/working/dogs-vs-cats/test1',
                                              x_col="file", y_col=None, target_size=(256, 256), color_mode='rgb', 
                                              batch_size=32, class_mode=None)

In [ ]:
model = tf.keras.Sequential([
#    tf.keras.layers.Conv2D(64,3,activation = 'relu'),
#    tf.keras.layers.Conv2D(64,3,activation = 'relu'),
#    tf.keras.layers.MaxPool2D(3),
    tf.keras.layers.Conv2D(32,3,activation = 'relu'),
    tf.keras.layers.Conv2D(32,3,activation = 'relu'),
    tf.keras.layers.MaxPool2D(3),
    tf.keras.layers.Conv2D(16,3,activation = 'relu'),
    tf.keras.layers.Conv2D(16,3,activation = 'relu'),
    tf.keras.layers.MaxPool2D(3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64,activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32,activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation = 'sigmoid'),
])
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
history = model.fit(train_set,epochs = 15, batch_size = 200, validation_data = val_set)

In [ ]:
plt.figure(figsize = (10,6))
plt.plot(np.arange(1,16), history.history['accuracy'])
plt.plot(np.arange(1,16), history.history['val_accuracy'])

In [ ]:
predict = model.predict(test)
predict = predict.reshape(-1)
predict[predict > 0.5] = 1
predict[predict < 0.5] = 0

In [ ]:
subm = pd.DataFrame({'id': np.arange(1,predict.shape[0]+1), 'label': predict.astype(int)})
subm.to_csv('submission.csv', index=False)